<a href="https://colab.research.google.com/github/datascience-vivek/Bike-shearing-Demand/blob/main/Team_notebook_Bike_Sharing_Demand_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Seoul Bike Sharing Demand Prediction </u></b>

## <b> Problem Description </b>

### Currently Rental bikes are introduced in many urban cities for the enhancement of mobility comfort. It is important to make the rental bike available and accessible to the public at the right time as it lessens the waiting time. Eventually, providing the city with a stable supply of rental bikes becomes a major concern. The crucial part is the prediction of bike count required at each hour for the stable supply of rental bikes.


## <b> Data Description </b>

### <b> The dataset contains weather information (Temperature, Humidity, Windspeed, Visibility, Dewpoint, Solar radiation, Snowfall, Rainfall), the number of bikes rented per hour and date information.</b>


### <b>Attribute Information: </b>

* ### Date : year-month-day
* ### Rented Bike count - Count of bikes rented at each hour
* ### Hour - Hour of he day
* ### Temperature-Temperature in Celsius
* ### Humidity - %
* ### Windspeed - m/s
* ### Visibility - 10m
* ### Dew point temperature - Celsius
* ### Solar radiation - MJ/m2
* ### Rainfall - mm
* ### Snowfall - cm
* ### Seasons - Winter, Spring, Summer, Autumn
* ### Holiday - Holiday/No holiday
* ### Functional Day - NoFunc(Non Functional Hours), Fun(Functional hours)

In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd

from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Reading and understanding the data

In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/SeoulBikeData.csv', encoding= 'unicode_escape',parse_dates=True)

In [ ]:
# Exploring the first five rows of the dataset
df.head()

In [ ]:
# Exploring the last five rows of the dataset
df.tail()

In [ ]:
# Inspecting the length and number of columns of the dataset
df.shape

In [ ]:
# Checking null values and data types of each columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       8760 non-null   object 
 1   Rented Bike Count          8760 non-null   int64  
 2   Hour                       8760 non-null   int64  
 3   Temperature(°C)            8760 non-null   float64
 4   Humidity(%)                8760 non-null   int64  
 5   Wind speed (m/s)           8760 non-null   float64
 6   Visibility (10m)           8760 non-null   int64  
 7   Dew point temperature(°C)  8760 non-null   float64
 8   Solar Radiation (MJ/m2)    8760 non-null   float64
 9   Rainfall(mm)               8760 non-null   float64
 10  Snowfall (cm)              8760 non-null   float64
 11  Seasons                    8760 non-null   object 
 12  Holiday                    8760 non-null   object 
 13  Functioning Day            8760 non-null   objec

In [ ]:
# duplicate
print(df[df.duplicated()].sum())

In [ ]:
# Finding statistical measures of numerical columns
df.describe()

In [ ]:
## Extracting month and year from 'Date' column

df['Date'] = pd.to_datetime(df['Date'])
df['year'] = pd.DatetimeIndex(df['Date']).year
df['month'] = pd.DatetimeIndex(df['Date']).month

In [ ]:
# Dropping the date column
df.drop('Date',axis = 1,inplace = True)

In [ ]:
#Checks the unique values of the newly formed columns
df['year'].unique()

In [ ]:
df['month'].unique()

In [ ]:
## Replacing month numbers with month names

df.replace({'month':{1: 'January',2: 'February',3:'March',4:'April',5:'May',6:'June',
                     7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}},inplace=True)

In [ ]:
df['Hour'].unique()

In [ ]:
df.head()

# Exploratory data analysis

**In this step we are going to discover pattens and to check assumptions with the help of visualization.** 

In [ ]:
# Extracting names of numerical columns from the data
numeric_features = df.describe().columns
numeric_features

In [ ]:
# Distplot and box plot of numeric features

for var in numeric_features:
    plt.figure(figsize=(12,6))
    plt.subplot(1, 2, 1)
    fig = sns.boxplot(y=df[var])
    fig.set_title('')
    fig.set_ylabel(var)
    
# Distplot  
    plt.subplot(1, 2, 2)
    fig = sns.distplot(df[var].dropna())
    fig.set_ylabel('frequency')
    fig.set_xlabel(var)

    plt.show()

**The distribution of temperature, humidity and dew point temperature are nearly normal.**

**The distribution of rented bike count and wind speed are slightly right skewed. Visibility column is severely left skewed.**

**Columns like snowfall, rainfall and solar radiation contains most of the values as zero.**

**There are less data for the year 2017.**




In [ ]:
# Relation of numeric features with Rented Bike Count
for col in numeric_features[1:]:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    feature = df[col]
    label = df['Rented Bike Count']
    correlation = feature.corr(label)
    plt.scatter(x=feature, y=label)
    plt.xlabel(col)
    plt.ylabel('Rented Bike Count')
    ax.set_title('Rented Bike Count vs ' + col + '- correlation: ' + str(correlation))
    z = np.polyfit(df[col], df['Rented Bike Count'], 1)
    y_hat = np.poly1d(z)(df[col])

    plt.plot(df[col], y_hat, "r--", lw=1)

plt.show()

**Features like hour, temperature and dew point temperature are highly correlated with the dependent variable as compared to other variables.**

**Humidity, snowfall and rainfall are negatively correlated with dependent variable with very low correlation coefficient.**

**The peak time for bike renting is during the working hours that is from morning 7am to 10am and from evening 4pm to 9pm.**



In [ ]:
# Square root transform of dependent variable to make the distribution normal
plt.figure(figsize=(7,7))
sns.distplot(np.sqrt(df['Rented Bike Count']),color="r")
plt.show()

In [ ]:
# Extracting names of categorical columns from the data
categorical_features = df.describe(include=['object','category']).columns
categorical_features

In [ ]:
# Bar plot of categorical features
for col in categorical_features:
    counts = df[col].value_counts().sort_index()
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    counts.plot.bar(ax = ax, color='steelblue')
    ax.set_title(col + ' counts')
    ax.set_xlabel(col) 
    ax.set_ylabel("Frequency")
plt.show()

In [ ]:
# Box plot of categorical features with respect to dependent variable
for col in categorical_features:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    df.boxplot(column = 'Rented Bike Count', by = col, ax = ax)
    ax.set_title('Label by ' + col)
    ax.set_ylabel("Rented Bike Count")
plt.show()

**Bike renting is less in winter season as compared to other seasons.**

**In holidays bike renting is slightly less.**

**There are very few peoples renting bikes in the month of December, January and february.**

In [ ]:
## Correlation matrix
plt.figure(figsize=(15,8))
correlation = df.corr()
sns.heatmap(abs(correlation), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Checking Multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

  # Calculating VIF
  vif = pd.DataFrame()
  vif["variables"] = X.columns
  vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

  return(vif)

In [ ]:
calc_vif(df[[i for i in df.describe().columns if i not in ['Rented Bike Count']]])

**As year has high VIF value, including it in our model results in unstable parameter estimate.**

In [ ]:
# VIF without year column
calc_vif(df[[i for i in df.describe().columns if i not in ['Rented Bike Count','year']]])

# Data preparation

In [ ]:
# Creating a list of columns
x = list(df.describe().columns)

In [ ]:
df.columns

In [ ]:
# Numeical features that will go to our model
numerical_features = [i for i in x if i not in ['Rented Bike Count','year','Rainfall(mm)','Snowfall (cm)']]

**As in snowfall and rainfall columns most of the values are zero and also correlation of them with the dependent variable is very less, we are not going to include them in our model building.**

In [ ]:
# Creating a copy so that we can make changes in the data during encoding.
df1 = df.copy()

In [ ]:
# Converting categorical features to numeric 
#label encoding
encoders_nums = {"Holiday":{"Holiday":1,"No Holiday":0} ,"Functioning Day":{"Yes":1,"No":0}}
df1= df1.replace(encoders_nums)

In [ ]:
# One hot encoding
df1 = pd.get_dummies(df1, columns=['Seasons','month'])

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
#Final feature list
features = numerical_features.copy()
features.extend(['Holiday','Functioning Day', 'Seasons_Autumn', 'Seasons_Spring', 'Seasons_Summer','Seasons_Winter',
                'month_April', 'month_August','month_December', 'month_February', 'month_January', 'month_July',
                 'month_June', 'month_March', 'month_May', 'month_November','month_October', 'month_September'])
features

In [ ]:
len(features)

**There are total 25 features and 1 dependent variable which we will use to fit our model.**

In [ ]:
#Normalizing the features
X = df1[features].apply(zscore)

In [ ]:
#Transforming dependent variable using squareroot transformation.
y = np.sqrt(df1['Rented Bike Count'])

In [ ]:
#Splitting the data into two parts where the test data contains 20% of orignal data.
X_train, X_test, y_train, y_test = train_test_split( X,y , test_size = 0.2, random_state = 5) 

In [ ]:
X_train.shape

In [ ]:
X_test.shape

# Linear Regression

**Model fitting**

In [ ]:
# Creating an instance for linear regression 
reg = LinearRegression()
# Model fitting
reg.fit(X_train, y_train)

In [ ]:
# Prediction
y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)

**Model evaluation**

In [ ]:
#Mean Squared Error and Root Mean Square Error for train data

MSE_train  = mean_squared_error((y_train)**2,(y_train_pred)**2)
print("Train MSE :" , MSE_train)

RMSE_train = np.sqrt(MSE_train)
print("Train RMSE :" ,RMSE_train)

#Mean Square Error and Root Mean Square Error for test data

MSE_test  = mean_squared_error((y_test)**2,(y_test_pred)**2)
print("Test MSE :" , MSE_test)

RMSE_test = np.sqrt(MSE_test)
print("Test RMSE :" ,RMSE_test)

In [ ]:
#r-square matrix for train and test datas
r2_train = r2_score((y_train)**2, (y_train_pred)**2)
r2_test = r2_score((y_test)**2, (y_test_pred)**2)
print("train r2 score :" , r2_train)
print("test r2 score :" , r2_test)

In [ ]:
# Calculation of adjusted r2
print("Adjusted_Train_R2_linear : ",1-(1-r2_score((y_train)**2, (y_train_pred)**2))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
print("Adjusted_Test_R2_linear : ",1-(1-r2_score((y_test)**2, (y_test_pred)**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

**As the accuracy score is very low, the model has high bias. To decrease the bias next we will try to do polynomial regression.** 

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(reg.coef_))], axis = 1)

In [ ]:
coef = coefficients.iloc[:,0]
val = coefficients.iloc[:,1]

In [ ]:
# creating the bar plot of feature importance
fig_dims = (4,8)
fig = plt.subplots(figsize=fig_dims)
plt.barh(coef, val, color ='maroon')
plt.show()

**Functioning day,hour and dew point temperature are affecting positively rented bike count the most.**

**Humidity negatively impacting most the dependent variable.**  

# Polynomial regression

In [ ]:
rmses = []
degrees = np.arange(1,5)
min_rmse, min_deg = 1e10, 0

for deg in degrees:

    # Train features
    poly_features = PolynomialFeatures(degree=deg, include_bias=True)
    X_poly_train = poly_features.fit_transform(X_train)

    # Linear regression
    poly_reg = LinearRegression()
    poly_reg.fit(X_poly_train, y_train)

    # Compare with test data
    X_poly_test = poly_features.fit_transform(X_test)
    poly_predict = poly_reg.predict(X_poly_test)
    poly_mse = mean_squared_error(y_test, poly_predict)
    poly_rmse = np.sqrt(poly_mse)
    rmses.append(poly_rmse)
    
    # Cross-validation of degree
    if min_rmse > poly_rmse:
        min_rmse = poly_rmse
        min_deg = deg

# Plot and present results
print('Best degree {} with RMSE {}'.format(min_deg, min_rmse))
        
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(degrees, rmses)
ax.set_yscale('log')
ax.set_xlabel('Degree')
ax.set_ylabel('RMSE')

**From the above plot, it is clear that best degree for polynomial regression is 3 for which RMSE = 5.12**

**model fitting**

In [ ]:
# Fitting three degree polynomial regression model 
poly = PolynomialFeatures(degree=3,include_bias=True)
X_train_trans = poly.fit_transform(X_train)
X_test_trans = poly.fit_transform(X_test)
lr = LinearRegression()
lr.fit(X_train_trans, y_train)

**model evaluation**

In [ ]:
# r2 score calculation
y_pred_train = lr.predict(X_train_trans)
print("Train r2 score:", r2_score((y_train)**2, (y_pred_train)**2))
y_pred_test = lr.predict(X_test_trans)
print("Test r2 score:",r2_score((y_test)**2, (y_pred_test)**2))

In [ ]:
# Calculation of adjusted r2
print("Adjusted_Train_R2_nonlinear : ",1-(1-r2_score((y_train)**2, (y_pred_train)**2))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
print("Adjusted_Test_R2_nonlinear : ",1-(1-r2_score((y_test)**2, (y_pred_test)**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
#MSE and RMSE for train data
MSE_tr = mean_squared_error((y_train)**2, (y_pred_train)**2)
print("Train MSE :" , MSE_tr)
print("Train RMSE :" , np.sqrt(MSE_tr))

In [ ]:
#MSE and RMSE for test data
MSE_te = mean_squared_error((y_test)**2, (y_pred_test)**2)
print("Test MSE :" , MSE_te)
print("Test RMSE :" , np.sqrt(MSE_te))

**At degree 3, the model is overfitting the data. We will now apply some regularization techniques to decrease the variance.**

# Ridge

In [ ]:
# Performing gridsearchcv on ridge regressor
ridge = Ridge()
parameters = {'alpha': [1e-3,1e-2,1e-1,1,5,10,20,30,100,.0014]}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=3)
ridge_regressor.fit(X_train_trans,y_train)

In [ ]:
# Best parameter from gridsearchcv
print("The best fit alpha value is found out to be :" ,ridge_regressor.best_params_)
print("\nUsing ",ridge_regressor.best_params_, " the negative mean squared error is: ", ridge_regressor.best_score_)

In [ ]:
# Prediction on train and test data
y_pred_train_ridge = ridge_regressor.predict(X_train_trans)
y_pred_test_ridge = ridge_regressor.predict(X_test_trans)

In [ ]:
#Calculation of MSE 
MSE_train_ridge  = mean_squared_error((y_train)**2, (y_pred_train_ridge)**2)
print("Train_MSE_ridge :" , MSE_train_ridge)
MSE_test_ridge  = mean_squared_error((y_test)**2, (y_pred_test_ridge)**2)
print("Test_MSE_ridge :" , MSE_test_ridge)


#Calculation of RMSE
RMSE_train_ridge = np.sqrt(MSE_train_ridge)
print("Train_RMSE_ridge :" ,RMSE_train_ridge)
RMSE_test_ridge = np.sqrt(MSE_test_ridge)
print("Test_RMSE_ridge :" ,RMSE_test_ridge)

In [ ]:
# Calculation of r2 score
r2_train_ridge = r2_score((y_train)**2, (y_pred_train_ridge)**2)
r2_test_ridge = r2_score((y_test)**2, (y_pred_test_ridge)**2)
print("r2_train_ridge :" ,r2_train_ridge)
print("r2_test_ridge :" ,r2_test_ridge)

In [ ]:
# Calculation of adjusted r2
print("Adjusted_Train_R2_ridge : ",1-(1-r2_score((y_train)**2, (y_pred_train_ridge)**2))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
print("Adjusted_Test_R2_ridge : ",1-(1-r2_score((y_test)**2, (y_pred_test_ridge)**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

# Lasso

In [ ]:
# Performing gridsearchcv on lasso regressor
lasso = Lasso()
parameters = {'alpha': [1e-3,1e-2,1e-1,1,5,10,20,30,100,.0014]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=3)
lasso_regressor.fit(X_train_trans, y_train)

In [ ]:
# Best parameter from gridsearchcv
print("The best fit alpha value is found out to be :" ,lasso_regressor.best_params_)
print("\nUsing ",lasso_regressor.best_params_, " the negative mean squared error is: ", lasso_regressor.best_score_)

In [ ]:
# Predictions
y_pred_lasso_test = lasso_regressor.predict(X_test_trans)
y_pred_lasso_train = lasso_regressor.predict(X_train_trans)

In [ ]:
#Calculation of MSE 
MSE_train_lasso  = mean_squared_error((y_train)**2, (y_pred_lasso_train)**2)
print("Train_MSE_lasso :" , MSE_train_lasso)
MSE_test_lasso  = mean_squared_error((y_test)**2, (y_pred_lasso_test)**2)
print("Test_MSE_lasso :" , MSE_test_lasso)


#Calculation of RMSE
RMSE_train_lasso = np.sqrt(MSE_train_lasso)
print("Train_RMSE_lasso :" ,RMSE_train_lasso)
RMSE_test_lasso = np.sqrt(MSE_test_lasso)
print("Test_RMSE_lasso :" ,RMSE_test_lasso)

In [ ]:
# Calculation of r2 score
r2_train_lasso = r2_score((y_train)**2, (y_pred_lasso_train)**2)
r2_test_lasso = r2_score((y_test)**2, (y_pred_lasso_test)**2)
print("r2_train_lasso :" ,r2_train_lasso)
print("r2_test_lasso :" ,r2_test_lasso)

In [ ]:
# Calculation of adjusted r2
print("Adjusted_Train_R2_lasso : ",1-(1-r2_score((y_train)**2, (y_pred_lasso_train)**2))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
print("Adjusted_Test_R2_lasso : ",1-(1-r2_score((y_test)**2, (y_pred_lasso_test)**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

# ElasticNet

In [ ]:
# Performing gridsearchcv on elesticnet regression
elastic = ElasticNet()
parameters = {'alpha': [1e-3,1e-2,1e-1,1,5,10,20,30,100,.0014],
              'l1_ratio':[0.3,0.4,0.5,0.6,0.7,0.8]}
elastic_regressor = GridSearchCV(elastic, parameters, scoring='neg_mean_squared_error', cv=3)
elastic_regressor.fit(X_train_trans, y_train)

In [ ]:
# Best parameter from gridsearchcv
print("The best fit parameters are found out to be :" ,elastic_regressor.best_params_)
print("\nUsing ",elastic_regressor.best_params_, " the negative mean squared error is: ", elastic_regressor.best_score_)

In [ ]:
# Prediction
y_pred_elasticnet_test = elastic_regressor.predict(X_test_trans)
y_pred_elasticnet_train = elastic_regressor.predict(X_train_trans)

In [ ]:
#Calculation of MSE 
MSE_train_elasticnet  = mean_squared_error((y_train)**2, (y_pred_elasticnet_train)**2)
print("Train_MSE_elasticnet :" , MSE_train_elasticnet)
MSE_test_elasticnet  = mean_squared_error((y_test)**2, (y_pred_elasticnet_test)**2)
print("Test_MSE_elasticnet :" , MSE_test_elasticnet)


#Calculation of RMSE
RMSE_train_elasticnet = np.sqrt(MSE_train_elasticnet)
print("Train_RMSE_elasticnet :" ,RMSE_train_elasticnet)
RMSE_test_elasticnet = np.sqrt(MSE_test_elasticnet)
print("Test_RMSE_elasticnet :" ,RMSE_test_elasticnet)

In [ ]:
# r2 score calculation
r2_train_elasticnet = r2_score((y_train)**2, (y_pred_elasticnet_train)**2)
r2_test_elasticnet = r2_score((y_test)**2, (y_pred_elasticnet_test)**2)
print("r2_train_elasticnet :" ,r2_train_elasticnet)
print("r2_test_elasticnet :" ,r2_test_elasticnet)

In [ ]:
# adjusted r2 score calculation
print("Adjusted_Train_R2_elasticnet : ",1-(1-r2_score((y_train)**2, (y_pred_elasticnet_train)**2))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
print("Adjusted_Test_R2_elasticnet : ",1-(1-r2_score((y_test)**2, (y_pred_elasticnet_test)**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

# Decision tree

In [ ]:
# Creating an instance for decision tree regressor
dt_model = DecisionTreeRegressor()

In [ ]:
# Defining parameter list to perform GridSearchCV
param_dict = {"criterion": ["mse", "mae"],
              "min_samples_split": [10, 20, 40],
              "max_depth": [2, 6, 8],
              "min_samples_leaf": [20, 40, 100],
              "max_leaf_nodes": [5, 20, 100],
              }
# GridSearchCV
dt_grid = GridSearchCV(estimator=dt_model,param_grid = param_dict, cv=5)

dt_grid.fit(X_train,y_train)

In [ ]:
# Getting best estimator from GridSearch
dt_grid.best_estimator_

In [ ]:
# Making predictions on train and test data

dt_train_preds = dt_grid.predict(X_train)
dt_test_preds = dt_grid.predict(X_test)

In [ ]:
#Calculation of MSE 
MSE_train_dt  = mean_squared_error((y_train)**2, (dt_train_preds)**2)
print("Train_MSE_dt :" , MSE_train_dt)
MSE_test_dt  = mean_squared_error((y_test)**2, (dt_test_preds)**2)
print("Test_MSE_dt :" , MSE_test_dt)


#Calculation of RMSE
RMSE_train_dt = np.sqrt(MSE_train_dt)
print("Train_RMSE_dt :" ,RMSE_train_dt)
RMSE_test_dt = np.sqrt(MSE_test_dt)
print("Test_RMSE_dt :" ,RMSE_test_dt)

In [ ]:
#r-square matrix for train and test datas
r2_train_dt = r2_score((y_train)**2, (dt_train_preds)**2)
r2_test_dt = r2_score((y_test)**2, (dt_test_preds)**2)
print("train r2 score dt :" , r2_train_dt)
print("test r2 score dt :" , r2_test_dt)

In [ ]:
# Calculation of adjusted r2
print("Adjusted_Train_R2_dt : ",1-(1-r2_score((y_train)**2, (dt_train_preds)**2))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
print("Adjusted_Test_R2_dt : ",1-(1-r2_score((y_test)**2, (dt_test_preds)**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
features = X_train.columns
importances = dt_grid.best_estimator_.feature_importances_
indices = np.argsort(importances)

In [ ]:
# Plotting feature importance
fig_dims = (4,8)
fig = plt.subplots(figsize=fig_dims)
plt.title('Feature Importance')
plt.barh(range(len(indices)), importances[indices], color='red', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')

plt.show()

# Random forest

In [ ]:
# Hyperparameter Grid
param_dict = { "n_estimators"      : [10,30,100,200],
               "min_samples_split" : [2,4,8],
               "max_depth"         : [5,10,20,30]
             }

In [ ]:
# Creating an instance of the RandomForestRegressor
rf_model = RandomForestRegressor()

# Grid search
rf_grid = GridSearchCV(estimator=rf_model,
                       param_grid = param_dict,
                       cv = 3)

rf_grid.fit(X_train,y_train)

In [ ]:
# Getting best estimator from GridSearch
rf_grid.best_estimator_

In [ ]:
# Making predictions on train and test data

rf_train_preds = rf_grid.predict(X_train)
rf_test_preds = rf_grid.predict(X_test)

In [ ]:
#Calculation of MSE 
MSE_train_rf  = mean_squared_error((y_train)**2, (rf_train_preds)**2)
print("Train_MSE_rf :" , MSE_train_rf)
MSE_test_rf  = mean_squared_error((y_test)**2, (rf_test_preds)**2)
print("Test_MSE_rf :" , MSE_test_rf)


#Calculation of RMSE
RMSE_train_rf = np.sqrt(MSE_train_rf)
print("Train_RMSE_rf :" ,RMSE_train_rf)
RMSE_test_rf = np.sqrt(MSE_test_rf)
print("Test_RMSE_rf :" ,RMSE_test_rf)

In [ ]:
#r-square matrix for train and test datas
r2_train_rf = r2_score((y_train)**2, (rf_train_preds)**2)
r2_test_rf = r2_score((y_test)**2, (rf_test_preds)**2)
print("train r2 score rf :" , r2_train_rf)
print("test r2 score rf :" , r2_test_rf)

In [ ]:
# Calculation of adjusted r2
print("Adjusted_Train_R2_rf : ",1-(1-r2_score((y_train)**2, (rf_train_preds)**2))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
print("Adjusted_Test_R2_rf : ",1-(1-r2_score((y_test)**2, (rf_test_preds)**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
features = X_train.columns
importances = rf_grid.best_estimator_.feature_importances_
indices = np.argsort(importances)

In [ ]:
# Plotting feature importance
fig_dims = (4,8)
fig = plt.subplots(figsize=fig_dims)
plt.title('Feature Importance')
plt.barh(range(len(indices)), importances[indices], color='red', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')

plt.show()

# Gradient boosting

In [ ]:
# Defining parameter list for GridSearch
param_dict= {'learning_rate' : [0.01,0.02,0.03,0.04],
              'subsample'    : [0.9, 0.5, 0.2, 0.1],
              'n_estimators' : [100,300,500],
              'max_depth'    : [4,6,8,10]
            }

In [ ]:
# Create an instance of the RandomForestClassifier
gb_model = GradientBoostingRegressor()

# Grid search
gb_grid = GridSearchCV(estimator=gb_model,
                       param_grid = param_dict,
                       cv = 2, n_jobs=-1)

gb_grid.fit(X_train,y_train)

In [ ]:
# Getting best estimator from GridSearchCV
gb_grid.best_estimator_

In [ ]:
# Making predictions on train and test data

gb_train_preds = gb_grid.predict(X_train)
gb_test_preds = gb_grid.predict(X_test)

In [ ]:
#Calculation of MSE 
MSE_train_gb  = mean_squared_error((y_train)**2, (gb_train_preds)**2)
print("Train_MSE_gb :" , MSE_train_gb)
MSE_test_gb  = mean_squared_error((y_test)**2, (gb_test_preds)**2)
print("Test_MSE_gb :" , MSE_test_gb)


#Calculation of RMSE
RMSE_train_gb = np.sqrt(MSE_train_gb)
print("Train_RMSE_gb :" ,RMSE_train_gb)
RMSE_test_gb = np.sqrt(MSE_test_gb)
print("Test_RMSE_gb :" ,RMSE_test_gb)

In [ ]:
#r-square matrix for train and test datas
r2_train_gb = r2_score((y_train)**2, (gb_train_preds)**2)
r2_test_gb = r2_score((y_test)**2, (gb_test_preds)**2)
print("train r2 score gb :" , r2_train_gb)
print("test r2 score gb :" , r2_test_gb)

In [ ]:
# Calculation of adjusted r2
print("Adjusted_Train_R2_gb : ",1-(1-r2_score((y_train)**2, (gb_train_preds)**2))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
print("Adjusted_Test_R2_gb : ",1-(1-r2_score((y_test)**2, (gb_test_preds)**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
features = X_train.columns
importances = gb_grid.best_estimator_.feature_importances_
indices = np.argsort(importances)

In [ ]:
# Plotting feature importance
fig_dims = (4,8)
fig = plt.subplots(figsize=fig_dims)
plt.title('Feature Importance')
plt.barh(range(len(indices)), importances[indices], color='red', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')

plt.show()

# XGboost

In [ ]:
# Defining parameter list for GridSearch
param_dict = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

In [ ]:
# Creating an instance for XGBoost Regressor
xgb_model = xgb.XGBRegressor(seed = 20)

In [ ]:
# Grid search
xgb_grid = GridSearchCV(estimator=xgb_model, 
                   param_grid=param_dict,
                   scoring='neg_mean_squared_error',cv=3, 
                   verbose=1)
xgb_grid.fit(X_train,y_train)


In [ ]:
# Getting best estimator from GridSearchCV
xgb_grid.best_estimator_

In [ ]:
# Making predictions on train and test data

xgb_train_preds = xgb_grid.predict(X_train)
xgb_test_preds = xgb_grid.predict(X_test)

In [ ]:
#Calculation of MSE 
MSE_train_xgb  = mean_squared_error((y_train)**2, (xgb_train_preds)**2)
print("Train_MSE_xgb :" , MSE_train_xgb)
MSE_test_xgb  = mean_squared_error((y_test)**2, (xgb_test_preds)**2)
print("Test_MSE_xgb :" , MSE_test_xgb)


#Calculation of RMSE
RMSE_train_xgb = np.sqrt(MSE_train_xgb)
print("Train_RMSE_xgb :" ,RMSE_train_xgb)
RMSE_test_xgb = np.sqrt(MSE_test_xgb)
print("Test_RMSE_xgb :" ,RMSE_test_xgb)

In [ ]:
#r-square matrix for train and test datas
r2_train_xgb = r2_score((y_train)**2, (xgb_train_preds)**2)
r2_test_xgb = r2_score((y_test)**2, (xgb_test_preds)**2)
print("train r2 score xgb :" , r2_train_xgb)
print("test r2 score xgb :" , r2_test_xgb)

In [ ]:
# Calculation of adjusted r2
print("Adjusted_Train_R2_xgb : ",1-(1-r2_score((y_train)**2, (xgb_train_preds)**2))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
print("Adjusted_Test_R2_xgb : ",1-(1-r2_score((y_test)**2, (xgb_test_preds)**2))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
features = X_train.columns
importances = xgb_grid.best_estimator_.feature_importances_
indices = np.argsort(importances)

In [ ]:
# Plotting feature importance
fig_dims = (4,8)
fig = plt.subplots(figsize=fig_dims)
plt.title('Feature Importance')
plt.barh(range(len(indices)), importances[indices], color='red', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')

plt.show()

# Conclusion

* With accuracy of 58% linear regression model has high bias.

* Non linear regression model(degree 3) with regularization gives good accuracy score of 83% on training data and 81% on testing data.

* Decision tree model gives accuracy score of 81% on training data whereas 78% accuracy on testing data.

* All the 3 different ensemble methods are overfitting the data with almost 10% difference between train and test accuracy.

* Out of all ensemble methods used gradient boosting and XGBoost giving slightly good result with training accuracy of 98% and testing accuracy of 88%.

* Temperature, hour and humidity are the three most important features given by all the models except XGBoost.